# 3 Selection and assessment of existing GSM
In this section the published iSynCJ816 model will be assessed. The heterologous pathways for production of p-coumaric acid will be introduced here, such that this assessment will be the reference to which the improvements of computer-aided cell factory engineering will be compared.

## 3.1 Reconstruction of existing GSM

In [1]:
import pytest
import hashlib
import numpy as np
from cobra.io import read_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite

model = read_sbml_model('iSynCJ816.xml')

#adding the metabolites (from KEGG, according to the article)
trans_Cinnamate = Metabolite(id = 'TRCI', compartment='c', name='trans-Cinnamate', formula='C9H8O2')
p_Coumaric_acid = Metabolite(id = 'PCOU', compartment='c', name='p-Coumaric acid', formula='C9H8O3')

#adding the reactions
phenylalanine_ammonia_lyase = Reaction('PAL')
trans_cinnamate_4_monooxygenase = Reaction('CYP73A')

#adding the metabolites into the reactions
phenylalanine_ammonia_lyase.add_metabolites(({model.metabolites.phe__L_c : -1,  
                              trans_Cinnamate: 1, 
                              model.metabolites.nh3_c: 1, 
                            }))
trans_cinnamate_4_monooxygenase.add_metabolites(({model.metabolites.o2_c : -1,
                                                  trans_Cinnamate: -1,
                                                  model.metabolites.nadph_c : -1,
                                                  p_Coumaric_acid: 1,
                                                  model.metabolites.h2o_c: 1,
                                                  model.metabolites.nadph_c : 1,
                                                 }))

#Adding the reactions to the model:
model.add_reactions([phenylalanine_ammonia_lyase])
model.add_reactions([trans_cinnamate_4_monooxygenase])

model.add_boundary(model.metabolites.TRCI, type='demand')
model.add_boundary(model.metabolites.PCOU, type='demand')

Reaction identifier,DM_PCOU
Name,p-Coumaric acid demand
Memory address,0x018dc31563c8
Stoichiometry,PCOU --> p-Coumaric acid -->
GPR,
Lower bound,0
Upper bound,1000.0


## 3.2 Study of important fluxes

### 3.2.1 p-coumaric production analysis

p-coumaric range of production is expressed as mmol p-coumaric/g(DW)h. Using experimental data, we can establish the bounds for CYP73A reaction. Gao et al. (2021) performed different Synechocystis sp. PCC 6803 tests, where the cyanobacteria were cultured during 7 days (168 hours) reaching a OD730 of 1,4 and then the concentration of p-coumaric acid measured, with results in the range 130-200 mg/L. The DW concentration of bacteria can be calculated as 148 mg/L/OD730 (Du, 2016). Calculations to obtain the final values and results can be seen below.

In [2]:
DW_factor = 0.148 #g/L/OD730
OD730 = 1.24 
DW_concentration = DW_factor * OD730
print('DW_concentration =', DW_concentration, 'g(DW)/L')
Time = 168 #hours
MW = 164 #g/mol
Coumaric_max_g = 0.200 #g/l
Coumaric_min_g = 0.130 #g/L
Coumaric_max_mmol = Coumaric_max_g / MW * 1000 
print('Coumaric_max_mmol/L', Coumaric_max_mmol, 'mmol/L')
Coumaric_min_mmol = Coumaric_min_g / MW * 1000 
print('Coumaric_min_mmol/L', Coumaric_min_mmol, 'mmol/L')
Productivity_max=Coumaric_max_mmol/DW_concentration/Time
Productivity_min=Coumaric_min_mmol/DW_concentration/Time
print('Productivity_max =', Productivity_max, 'mmol/g(DW)/h') 
print('Productivity_min =', Productivity_min, 'mmol/g(DW)/h') 

DW_concentration = 0.18352 g(DW)/L
Coumaric_max_mmol/L 1.2195121951219512 mmol/L
Coumaric_min_mmol/L 0.7926829268292683 mmol/L
Productivity_max = 0.03955427834263397 mmol/g(DW)/h
Productivity_min = 0.025710280922712082 mmol/g(DW)/h


In [3]:
with model as model:
    model.objective=model.reactions.CYP73A
    maximum_growth_hetero = model.optimize().objective_value #flux
    print('Maximum p-coumaric =', maximum_growth_hetero, 'mmol/g(DW)/h')

Maximum p-coumaric = 3.4849238349761794e-05 mmol/g(DW)/h


As we can see, the difference between the experimental p-coumaric production and the one predicted by the model is very high. Thus,from this comparation can be concluded that the model needs a lot of manual curing and optimization to be able to predict the p-coumaric production and be realistic. However, in the next analysis we will use the predicted value as reference to see if p-coumaric production can be improved by genome editing techniques. 

### 3.2.2 Photon exchange analysis

In [4]:
with model as model:
    model.objective=model.reactions.EX_photon_e
    maximum_growth_hetero = model.optimize().objective_value #flux
    print('Maximum photon exchange =', maximum_growth_hetero, 'mmol/g(DW)/h')
    print('Photon exchange bounds =', model.reactions.EX_photon_e.bounds)

Maximum photon exchange = 0.0 mmol/g(DW)/h
Photon exchange bounds = (0.0, 0.0)


Photons are essential for any photosynthetic microorganism. The model doesn't consider photon exchange, as the bounds are set to 0. A bioenergetic assesment carried out by Touloupakis et al. (2015) showed that photon exchange flux in *Synechocystis* is ranged up to 95 mmol/g(DW)/h, being optimal at 50 mmol/g(DW)/h. Bounds are set as [-50, 0] for autotrophic conditions. 

### 3.2.3 Glucose update analysis

In [5]:
with model as model:
    model.objective=model.reactions.BIOMASS_Ec_SynAuto_1
    maximum_growth_hetero = model.optimize().objective_value #flux
    glucose=model.reactions.EX_glc__D_e.flux
    print('Maximum glucose exchange =', glucose, 'mmol/g(DW)/h')
    print('Photon exchange bounds =', model.reactions.EX_glc__D_e.bounds)

with model as model:
    model.objective=model.reactions.BIOMASS_Ec_SynMixo_1
    maximum_growth_hetero = model.optimize().objective_value #flux
    glucose=model.reactions.EX_glc__D_e.flux
    print('Maximum glucose exchange =', glucose, 'mmol/g(DW)/h')
    print('Photon exchange bounds =', model.reactions.EX_glc__D_e.bounds)

Maximum glucose exchange = -0.85 mmol/g(DW)/h
Photon exchange bounds = (-0.85, 999999.0)
Maximum glucose exchange = -0.85 mmol/g(DW)/h
Photon exchange bounds = (-0.85, 999999.0)


Glucose uptake is done under heterotrophic and mixotrophic mode. Our model shows a glucose uptake rate of 0.85 mmol/g(DW)/h for both heterotrophic and mixotrophic modes. Metabolic flux analysis carried out by Yang (2002) shows that 0.85 and 0.38 are the rates for heterotrophic and mixotrophic modes, respectively. 

### 3.2.4 CO2 fixation (RuBisCO) analysis

In [6]:
with model as model:
    model.objective=model.reactions.RBPC_1
    co2fix = model.optimize().objective_value #flux
    print('Maximum CO2 fixation rate =', co2fix, 'mmol/g(DW)/h')
    print('CO2 exchange bounds =', model.reactions.RBPC_1.bounds)


Maximum CO2 fixation rate = 71435.92738095239 mmol/g(DW)/h
CO2 exchange bounds = (0.0, 999999.0)


CO2 fixation is the main process that photosynthetic microorganisms use to obtain carbon. CO2 is incorporated to ribulose-1,5-biphosphate, in a reaction catalyzed by RuBisCO (RBPC_1). Thus, we use flux trhough RuBisCO as a way to measure CO2 fixation. The model predicts a optimized value that is too high, so bounds must be changed to perform model analysis. A characterization of *Synechocystis* sp- PC6803 carried out by Zavřel et al. (2017), showed CO2 fixation rate range to be 2.97 - 3.54 mmol(CO2)/g(DW)/h. Bounds are set as [2.97, 3.54] for autotrophic and mixotrophic conditions.


## 3.2. Autotrophic, heterotrophic and mixotrophic comparation

It has already been explained in the literature review of the cell factory section how *Synechocystis* sp. PCC6803 can switch between autotrophic, heterotrophic and mixotrophic metabolism. Our model considers this three metabolisms, and has a specific biomass reaction for each of them. By setting each biomass reaction as objective and optimizing the flux through it the model will measure the rest ofthe metabolic flux as autotrophic, mixotrophic or heterotrophic behaviour.
In addition, some other paramenters are modified to optimize the metabolism mode simulation. The model doesn't consider flux through CO2 fixation and photon exchange, so specific RBPC_1 and EX_photon_e bounds are established using literature data.

In [7]:
#Autotrophic
with model as model:
    model.reactions.EX_photon_e.bounds = -50, 0
    model.reactions.RBPC_1.bounds = 2.97, 3.54
    model.objective=model.reactions.BIOMASS_Ec_SynAuto_1
    maximum_growth_hetero = model.optimize().objective_value #flux
    print('Maximum growth =', maximum_growth_hetero, 'h-1')
    p_coumaric_production = model.reactions.CYP73A.flux
    print('Maximum productivity =', p_coumaric_production, 'mmol p-coumaric/gDW*h')
    maximum_yield_co2 = p_coumaric_production / (1*(model.reactions.RBPC_1.flux)) #yield calculations
    print('Maximum theoretical yield on co2 =', maximum_yield_co2, 'mmol-p_coumaric/mmol-co2')
    cmolmaximum_yield_co2 = p_coumaric_production*9 / (1*(model.reactions.RBPC_1.flux)) #yield calculations
    print('C-mmol Maximum theoretical yield on co2 =', cmolmaximum_yield_co2, 'c-mmol-p_coumaric/c-mmol-co2')

Maximum growth = 0.17433033139528287 h-1
Maximum productivity = 7.786812912192326e-05 mmol p-coumaric/gDW*h
Maximum theoretical yield on co2 = 2.199664664461109e-05 mmol-p_coumaric/mmol-co2
C-mmol Maximum theoretical yield on co2 = 0.0001979698198014998 c-mmol-p_coumaric/c-mmol-co2


In [8]:
#Mixotrophic
with model as model:
    model.reactions.RBPC_1.bounds = 2.97, 3.54
    model.objective=model.reactions.BIOMASS_Ec_SynMixo_1
    maximum_growth_mixo = model.optimize().objective_value #flux
    print('Maximum growth =', maximum_growth_mixo, 'h-1')
    p_coumaric_production = model.reactions.CYP73A.flux
    print('Maximum productivity =', p_coumaric_production, 'mmol p-coumaric/gDW*h')
    maximum_yield_co2 = p_coumaric_production / (1*(model.reactions.RBPC_1.flux)) #yield calculations
    print('Maximum theoretical yield on co2 =', maximum_yield_co2, 'mmol-p_coumaric/mmol-co2')
    maximum_yield_glucose = p_coumaric_production / (-1*(model.reactions.EX_glc__D_e.flux)) #yield calculations
    print('Maximum theoretical yield on glucose =', maximum_yield_glucose, 'mmol-p-coumaric/mmol-glucose')
    cmolmaximum_yield_co2 = p_coumaric_production*9 / (1*(model.reactions.RBPC_1.flux)) #yield calculations
    print('C-mmol Maximum theoretical yield on co2 =', cmolmaximum_yield_co2, 'c-mmol-p_coumaric/c-mmol-co2')
    cmolmaximum_yield_glucose = p_coumaric_production*9 / (-1*(model.reactions.EX_glc__D_e.flux)*6) #yield calculations
    print('C-mmol Maximum theoretical yield on glucose =', cmolmaximum_yield_glucose, 'c-mmol-p-coumaric/c-mmol-glucose')

Maximum growth = 0.07802010063372732 h-1
Maximum productivity = 3.4849238350415784e-05 mmol p-coumaric/gDW*h
Maximum theoretical yield on co2 = 1.173375028633528e-05 mmol-p_coumaric/mmol-co2
Maximum theoretical yield on glucose = 4.099910394166563e-05 mmol-p-coumaric/mmol-glucose
C-mmol Maximum theoretical yield on co2 = 0.00010560375257701752 c-mmol-p_coumaric/c-mmol-co2
C-mmol Maximum theoretical yield on glucose = 6.149865591249844e-05 c-mmol-p-coumaric/c-mmol-glucose


In [9]:
#Heterotrophic
with model as model:
    model.reactions.RBPC_1.bounds = 0, 0
    model.objective=model.reactions.BIOMASS_Ec_SynHetero_1
    maximum_growth_hetero = model.optimize().objective_value #flux
    print('Maximum growth =', maximum_growth_hetero, 'h-1')
    p_coumaric_production = model.reactions.CYP73A.flux
    print('Maximum productivity =', p_coumaric_production, 'mmol p-coumaric/gDW*h')
    maximum_yield_glucose = p_coumaric_production / (-1*(model.reactions.EX_glc__D_e.flux)) #yield calculations
    print('Maximum theoretical yield on glucose =', maximum_yield_glucose, 'mmol-p-coumaric/mmol-glucose')
    cmolmaximum_yield_glucose = p_coumaric_production*9 / (-1*(model.reactions.EX_glc__D_e.flux)*6) #yield calculations
    print('C-mmol Maximum theoretical yield on glucose =', cmolmaximum_yield_glucose, 'c-mmol-p-coumaric/c-mmol-glucose')

Maximum growth = 0.07802010063372736 h-1
Maximum productivity = 3.484923835011394e-05 mmol p-coumaric/gDW*h
Maximum theoretical yield on glucose = 4.099910394131052e-05 mmol-p-coumaric/mmol-glucose
C-mmol Maximum theoretical yield on glucose = 6.149865591196579e-05 c-mmol-p-coumaric/c-mmol-glucose


### 3.2.1 Summary of growth model comparison

|  | Max. growth (h-1) | Max. productivity (mmol/g(DW)/h)| Max. yield on glucose | Max yield on CO2 | C-mmol max yield on glucose | C-mmol max yield on CO2 |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| Autotrophic | 0.17 | 7.78E-5| --- | 2.62E-5 | --- | 2.35E-4 |
| Mixootrophic | 0.078 |3.48E-5 | 4.09E-5 | 1.17E-5 | 6.14E-5 | 1.05E-4 |
| Heterotrophic | 0.078 | 3.48E-5 | 4.09E-5 | --- |  6.14E-5 | --- |


Results are very similar in the three conditions. Yang et al. (2002) reported a maximum heterotrophic and mixotrophic growth rate of 0.076 and 0.059 h-1, respectively. For the autotrophic growth rate, van Alphen et al. (2018) reported 0.16 h-1 under optimized light conditions, although standard conditions show 0.09 h-1. Thus, growth rates predicted by our models are in the range of literature data and are valid. 

p-coumaric production is equal to the result obtained for the optimization of CYP73A flux. In this sense, our model predicts a low p-coumaric production but this production remains constant when the biomass growth is optimized. Thus, the product can be biosynthesized under optimal growth conditions. 

Finally, C-mmol yields on glucose and CO2 show that CO2 asimilation leads to a higher c-mmol production of p-coumaric acid. Autotrophic mode is feasible in terms of sustainability, as more carbon from the external source end up in p-coumaric acid molecule than in heterotrophic or mixotrophic modes. 